In [10]:
import os
import openai

from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")

In [21]:

SANDBOX_DIR = "/Users/aidand/dev/babyagi/sandbox"
if not os.path.exists(SANDBOX_DIR):
    os.makedirs(SANDBOX_DIR)

def create_file(path, content):
    with open(os.path.join(SANDBOX_DIR, path), "w") as f:
        f.write(content)
    print(f"Created file {os.path.join(SANDBOX_DIR, path)}")

# Does the same thing as create_file - but nice to have a separate function for updating files
# So the LLM has the option to update files if it wants to - if that makes more sense than creating a new file
def update_file(path, content):
    with open(os.path.join(SANDBOX_DIR, path), "w") as f:
        f.write(content)
    print(f"Updated file {os.path.join(SANDBOX_DIR, path)}")

def delete_file(path):
    os.remove(os.path.join(SANDBOX_DIR, path))
    print(f"Deleted file {os.path.join(SANDBOX_DIR, path)}")

def make_dir(path):
    os.makedirs(os.path.join(SANDBOX_DIR, path))
    print(f"Created directory {os.path.join(SANDBOX_DIR, path)}")

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "create_file",
            "description": "Create a file with the given name and content",
            "parameters": {
                "type": "object",
                "properties": {
                    "path": {
                        "type": "string",
                        "description": "The relative path to the file to create",
                    },
                    "content": {
                        "type": "string",
                        "description": "The content of the file to create",
                    },
                },
                "required": ["path", "content"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "update_file",
            "description": "Update a file with the given name and content",
        },
        "parameters": {
            "type": "object",
            "properties": {
                "path": {
                    "type": "string",
                    "description": "The relative path to the file to update",
                },
                "content": {
                    "type": "string",
                    "description": "The content of the file to update",
                },
            },
            "required": ["path", "content"],
        },
    },
    {
        "type": "function",
        "function": {
            "name": "delete_file",
            "description": "Update a file with the given name and content",
        },
        "parameters": {
            "type": "object",
            "properties": {
                "path": {
                    "type": "string",
                    "description": "The relative path to the file to delete",
                },
            },
            "required": ["path"],
        },
    },
    {
        "type": "function",
        "function": {
            "name": "make_dir",
            "description": "Create a directory with the given name",
            "parameters": {
                "type": "object",
                "properties": {
                    "path": {
                        "type": "string",
                        "description": "The relative path to the directory to create",
                    },
                },
                "required": ["path"],
            },
        },
    },
]

In [29]:
from openai import OpenAI
from pydantic import BaseModel
import json

PROGRAM_OBJECTIVE="download a PDF from the internet and convert it to a text file."

CODER_AGENT_SYSTEM_PROMPT="""
You are a software engineer who is writing code to build a small python codebase that will {PROGRAM_OBJECTIVE}.
"""

from typing import List

class Plan(BaseModel):
    steps: List[str]

client = OpenAI(api_key=OPENAI_API_KEY)

# Coding agent creates a step by step plan
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": CODER_AGENT_SYSTEM_PROMPT},
        {"role": "user", "content": """
         Create a step by step plan to complete the task of creating a codebase that will {PROGRAM_OBJECTIVE}.
         You have 4 different operations you can perform. create_file(path, content), update_file(path, content), delete_file(path), make_dir(path).
         Limit your step by step plan to only these operations.
         Please include a README.md file in the root of the codebase that describes the codebase and how to run it.
         Please include a requirements.txt file in the root of the codebase that describes the dependencies of the codebase.
         """},
    ],
    tools=TOOLS,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "Plan",
            "description": "A plan to complete the task of creating a codebase that will {PROGRAM_OBJECTIVE}.",
            "strict": True,
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {"type": "array", "items": {"type": "string"}},
                },
                "required": ["steps"],
                "additionalProperties": False,
            },
        },
    },
)
plan = json.loads(response.choices[0].message.content)
print("Coder Agent - Plan")
for step in plan["steps"]:
    print(step)
print("-" * 100)

# Being more concrete
# print("Coder Agent - Being more concrete")
# for step in plan["steps"]:
#     response = client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=[
#             {"role": "system", "content": CODER_AGENT_SYSTEM_PROMPT},
#             {"role": "user", "content": """
#              You have come up with the following plan: {json.dumps(plan)}.
#              Please be more concrete about the steps you will take to complete the task of creating a codebase that will {PROGRAM_OBJECTIVE}.
#              For example:

#              Step: Create a README.md file in the 'my_project' directory with content describing the project and how to run it
#              """},
#         ],
#     )
#     print(response.choices[0].message.content)

# Coding agent executes the plan
print("Coder Agent - Executing Plan")
for step in plan["steps"]:
    prompt = f"""
        You have 4 different operations you can perform. create_file(path, content), update_file(path, content), delete_file(path), make_dir(path).
        Please perform the following operation: {step}
        """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": CODER_AGENT_SYSTEM_PROMPT},
            {"role": "user", "content": prompt},
        ],
        tools=TOOLS,
    )
    print(response.choices[0].message)

# Review agent reviews the code


Coder Agent - Plan
Create a directory for the codebase.
Create a README.md file in the root of the codebase that describes the project and how to run it.
Create a requirements.txt file in the root of the codebase that lists the necessary dependencies for the project.
Create the main Python module file that contains the core functionality of the codebase.
If there are any additional modules or scripts, create those files accordingly.
Test the code to ensure it works as intended.
----------------------------------------------------------------------------------------------------
Coder Agent - Executing Plan
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_cN3nv4Sm7doybM1bIkEtr0a5', function=Function(arguments='{"path":"codebase/README.md","content":"# Codebase\\n\\nThis is a codebase for a small Python project."}', name='create_file'), type='function')])
ChatCompletionMessage(content=Non